In [155]:
import sys
import os
import pandas as pd
import random

sys.path.append(os.path.abspath('../src'))

from models import Player, Team, Individual

In [156]:
players_csv = pd.read_csv('../data/players.csv')

In [157]:
players = []
for idx, row in players_csv.iterrows():
    p = Player(
        player_id=idx + 1,
        name=row['Name'],
        position=row['Position'],
        skill=row['Skill'],
        cost=row['Salary (€M)']
    )

    players.append(p)

print(f"Número de jogadores carregados: {len(players)}")

Número de jogadores carregados: 35


In [158]:
def create_valid_team(players):
    team = []
    used_ids = set()
    try:
        for pos, count in [('GK',1), ('DEF',2), ('MID',2), ('FWD',2)]:
            options = [p for p in players if p.position == pos and p.id not in used_ids]
            selected = random.sample(options, count)
            team.extend(selected)
            used_ids.update(p.id for p in selected)
        new_team = Team(team)
        if new_team.total_cost() <= 750:
            return new_team
        else:
            return create_valid_team(players) 
    except:
        return None

In [159]:
team = create_valid_team(players)
print(team.players)
print("Válida:", team.is_valid())
print("Custo total:", team.total_cost())
print("Skill média:", team.average_skill())

[Jordan Smith (GK) - Skill: 88 | €M100, Brayden Hughes (DEF) - Skill: 87 | €M100, Daniel Foster (DEF) - Skill: 90 | €M110, Austin Torres (MID) - Skill: 82 | €M80, Gavin Richardson (MID) - Skill: 87 | €M95, Adrian Collins (FWD) - Skill: 85 | €M90, Tyler Jenkins (FWD) - Skill: 80 | €M70]
Válida: True
Custo total: 645
Skill média: 85.57142857142857


In [160]:
def create_valid_individual(players):
    all_ids = set()
    teams = []
    for _ in range(5):
        team = None
        tries = 0
        while not team and tries < 10:
            team = create_valid_team([p for p in players if p.id not in all_ids])
            tries += 1
        if team:
            teams.append(team)
            all_ids.update(p.id for p in team.players)
    return Individual(teams) if len(teams) == 5 else None

In [161]:
ind = create_valid_individual(players)
if ind is not None:
    print("Indivíduo válido: ", ind.is_valid())
    print("Fitness (desvio padrão negativo):", ind.fitness())
else:
    print("⚠️ Não foi possível criar um indivíduo válido.")

Indivíduo válido:  True
Fitness (desvio padrão negativo): -1.0667516972911095
